<a href="https://colab.research.google.com/github/Mel-iza/Weni_XP_IA/blob/main/notebook/weni_xp_hands_on_ia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0. Exploração do problema

### 1. Instalação das bibliotecas e importação dos dados 


Instalação das bibliotecas que vamos utilizar:

In [ ]:
# Instalação das bibliotecas necessárias

!pip install scikit-learn==1.2.1
!pip install numpy==1.24.2
!pip install scipy==1.2.1 
!pip install pandas==1.5.3
!pip install Unidecode==1.3.6
!pip install nltk==3.8.1
!pip install spacy==3.5.0
!python -m spacy download pt_core_news_sm

Importação das bibliotecas e módulos

In [10]:
# Importando as bibliotecas

# Manipulação de dados
import pandas as pd
import numpy as np
import unicodedata
import unidecode
import re


# Machine Learning, vetorização e métricas
import sklearn
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegressionCV


# Tratamento e manipulação de texto
import nltk
import spacy
import pt_core_news_sm
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

import warnings
warnings.simplefilter("ignore")

----------------------

Importando os dados

In [ ]:
# Seu código aqui
# Importando o conjunto de dados

data = pd.read_csv('https://raw.githubusercontent.com/Mel-iza/Weni_XP_IA/main/data/train_dataset_1.csv?token=GHSAT0AAAAAABZRO4D4I4DF6WM22KDQ62NAZAHRWYQ')
data

In [31]:
#@title resultado esperado:

data =  pd.read_csv('/content/train_dataset_1.csv')
data

,text,label
0,"Olá, muito prazer! Sou o seu assistente virtua...",chatbot
1,"Fechado, vamos fazer assim. Você passa os dado...",chatbot
2,"Pra adiantar, vou precisar que você me fale qu...",chatbot
3,"Humm eu não entendi, desculpa.\n\nDigite 1 par...",chatbot
4,Pera só um instante que vou checar aqui se enc...,chatbot
...,...,...
1537,42,humano
1538,start,humano
1539,Quais são os riscos? 😷,humano
1540,Nao,humano


----------------

### 2. Conhecer e pré-processar os dados


Conferindo algumas informações básicas sobre o conjunto de dados

In [37]:
# Seu código aqui
# Mostrando algumas informações básicas do conjunto de dados



In [34]:
#@title resultado esperado:


print('linhas e colunas: ', data.shape)
print('\n')
data.info()

linhas e colunas:  (1542, 2)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1542 entries, 0 to 1541
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1542 non-null   object
 1   label   1542 non-null   object
dtypes: object(2)
memory usage: 24.2+ KB


Conferindo algumas linhas do conjunto de dados:
- primeiras linhas
- linhas escolhidas aleatoriament

In [ ]:
# Seu código aqui
# Mostrando as primeiras linhas do conjunto de dados




In [42]:
#@title Resultado esperado:

data.head()

,text,label
0,"Olá, muito prazer! Sou o seu assistente virtua...",chatbot
1,"Fechado, vamos fazer assim. Você passa os dado...",chatbot
2,"Pra adiantar, vou precisar que você me fale qu...",chatbot
3,"Humm eu não entendi, desculpa.\n\nDigite 1 par...",chatbot
4,Pera só um instante que vou checar aqui se enc...,chatbot


In [ ]:
# seu código aqui
# Mostrando algumas linhas do conjunto de dados de forma aleatoria



In [43]:
#@title Resultado próximo ao esperado

data.sample(10)

,text,label
1011,Não,humano
1095,"Sentir uma crise de tosse durante a madruga, p...",humano
590,Sim,humano
5,Você tem algum desses problemas de saúde?\n\n1...,chatbot
480,Do que você precisa agora? (Digite o número da...,chatbot
640,start,humano
553,16/01/1979,humano
641,Não,humano
803,ok. Obrigada!,humano
1520,start,humano


---------------

Vamos verificar a quantidade de dados nulos presente no conjunto de dados?


In [ ]:
# Seu código aqui
# Quantidade de dados nulos



In [44]:
#@title Resultado esperado:

data.isnull().sum()

text     0
label    0
dtype: int64

E agora, como fazer com que a máquina consiga compreender esses textos?. <br>

* `padronização`
* `removeção stop-words (a, e, de, o, etc)`
* `tokenização`

In [45]:
#Função para remover acentos e números

def remover_acentos_e_numeros(txt):
    nfkd = unicodedata.normalize('NFKD', txt)
    palavraSemAcento = u"".join([c for c in nfkd if not unicodedata.combining(c)])
    palavraSemAcento = re.sub(r"(@[A-Za]+)|([^A-Za-z \t])|(\w+:\/\/\S+)| ^rt|http.+?","", palavraSemAcento)
    palavraSemAcento = str(palavraSemAcento).lower()
    palavraSemAcento = re.sub(r"\d+", "", palavraSemAcento)
    palavraSemAcento = re.sub(r'  ', ' ', palavraSemAcento)
    palavraSemAcento = re.sub(r'compartilheversao para impressao comentarios',' ',
                                palavraSemAcento)
    return palavraSemAcento

In [61]:
#@title Resultado esperado:
# Aplicando a função de padronização no texto e colocando ele em uma nova coluna chamada 'text_clean'

data['texto_padronizado'] = [remover_acentos_e_numeros(i) for i in data['text']]
data.head(12)

,text,label,texto_padronizado
0,"Olá, muito prazer! Sou o seu assistente virtua...",chatbot,ola muito prazer sou o seu assistente virtual ...
1,"Fechado, vamos fazer assim. Você passa os dado...",chatbot,fechado vamos fazer assim voce passa os dados ...
2,"Pra adiantar, vou precisar que você me fale qu...",chatbot,pra adiantar vou precisar que voce me fale qua...
3,"Humm eu não entendi, desculpa.\n\nDigite 1 par...",chatbot,humm eu nao entendi desculpadigite para falar ...
4,Pera só um instante que vou checar aqui se enc...,chatbot,pera so um instante que vou checar aqui se enc...
5,Você tem algum desses problemas de saúde?\n\n1...,chatbot,voce tem algum desses problemas de saude diabe...
6,Infelizmente eu não consegui te entender. Você...,chatbot,infelizmente eu nao consegui te entender voce ...
7,Qual o seu primeiro nome?,chatbot,qual o seu primeiro nome
8,Você não possui nenhum pedido em aberto!,chatbot,voce nao possui nenhum pedido em aberto
9,Você está com algum dos sintomas abaixo?\n\n1....,chatbot,voce esta com algum dos sintomas abaixo febre ...


In [77]:
# Remover as stop words

stop_portugues = list(open(r'stopwords-pt.txt', encoding="UTF-8"))
stop_portugues= [i.replace('\n','') for i in stop_portugues] 
stop_portugues = [remover_acentos_e_numeros(i) for i in stop_portugues]

def remover_stop_words(x):
    return [w for w in x if w not in stop_portugues]

['a', 'acerca', 'adeus', 'agora', 'ainda', 'alem', 'algmas', 'algo', 'algumas', 'alguns', 'ali', 'alem', 'ambas', 'ambos', 'ano', 'anos', 'antes', 'ao', 'aonde', 'aos', 'apenas', 'apoio', 'apontar', 'apos', 'apos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aqui', 'aquilo', 'as', 'assim', 'atraves', 'atras', 'ate', 'ai', 'baixo', 'bastante', 'bem', 'boa', 'boas', 'bom', 'bons', 'breve', 'cada', 'caminho', 'catorze', 'cedo', 'cento', 'certamente', 'certeza', 'cima', 'cinco', 'coisa', 'com', 'como', 'comprido', 'conhecido', 'conselho', 'contra', 'contudo', 'corrente', 'cuja', 'cujas', 'cujo', 'cujos', 'custa', 'ca', 'da', 'daquela', 'daquelas', 'daquele', 'daqueles', 'dar', 'das', 'de', 'debaixo', 'dela', 'delas', 'dele', 'deles', 'demais', 'dentro', 'depois', 'desde', 'desligado', 'dessa', 'dessas', 'desse', 'desses', 'desta', 'destas', 'deste', 'destes', 'deve', 'devem', 'devera', 'dez', 'dezanove', 'dezasseis', 'dezassete', 'dezoito', 'dia', 'diante', 'direita', 'dispoe', 'dispoem', '

In [121]:


data['texto_padronizado'] = data['texto_padronizado'].str.split().apply(remover_stop_words).str.join(' ')

In [127]:
data.head(8)

,text,label,texto_padronizado
0,"Olá, muito prazer! Sou o seu assistente virtua...",chatbot,ola prazer assistente virtual papel ajudar ven...
1,"Fechado, vamos fazer assim. Você passa os dado...",chatbot,fechado vamos passa dados deixo dei cadastro p...
2,"Pra adiantar, vou precisar que você me fale qu...",chatbot,adiantar vou precisar fale email
3,"Humm eu não entendi, desculpa.\n\nDigite 1 par...",chatbot,humm entendi desculpadigite falar consultordig...
4,Pera só um instante que vou checar aqui se enc...,chatbot,pera instante vou checar encontro algum pedido...
5,Você tem algum desses problemas de saúde?\n\n1...,chatbot,algum problemas saude diabetes pressao alta pr...
6,Infelizmente eu não consegui te entender. Você...,chatbot,infelizmente consegui entender gostaria falar ...
7,Qual o seu primeiro nome?,chatbot,


### 3. Vamos modelar? Criação de um modelo de classificação

In [ ]:
LogisticRegressionCV()

In [ ]:
# Separando o texto em features e targets

X = df.text_clean
Y = df.label

In [ ]:
# Instanciando o CountVectorizer para o BOW (Bag of Words)

tokenizer = CountVectorizer(ngram_range=(1,1))

In [ ]:
# Tokenizando os dados + Bag Of Words

tokenizer.fit(X)
X_mat = tokenizer.transform(X)
X_df = pd.DataFrame(X_mat.todense())

In [ ]:
# Transformando o y em número para que o modelo consiga avaliar a classe. Escolhi o valor 0 para chatbot e 1 para humano
y = y.replace(['chatbot', 'humano'], [0, 1]).astype('category')


# Cross validation para avaliar a acurácia dos modelos.
for nome, modelo in modelos.items():
    accuracy = cross_val_score(modelo, X_df, y, cv = 3, scoring='accuracy').mean() # Definindo 3 folds e fazendo a média
    print(nome, accuracy)

### 4. Como saber se nosso modelo foi bem? Interpretação de métricas.

###. 5. Referências

* [Documentação Scikit Learn](https://scikit-learn.org/stable/)